In [1]:
import pandas as pd
import numpy as np

In [100]:
heart_data_path = './datasets/heart_data.csv'
heart_df = pd.read_csv(heart_data_path)

### Описание данных
1. age: возраст в годах
2. sex: пол
    - 1: мужской
    - 0: женский
3. cp: тип боли в груди
    - 0: типичная стенокардия
    - 1: нетипичная стенокардия
    - 2: боль, не связанная со стенокардией
    - 3: бессимптомная
4. trtbps: артериальное давление в покое в мм рт.ст.
5. chol: уровень холестерина в мг/дл
6. fbs: уровень сахара в крови натощак > 120 мг/дл
    - 1: True
    - 2: False
7. restecg: результаты электрокардиографии в состоянии покоя
    - 0: норма
    - 1: аномалия зубца ST-T (инверсия зубца T и/или подъем или спад сегмента ST > 0.05 мВ)
    - 2: указывает на вероятную или определенную гипертрофию левого желудочка по критериям Эстеса
8. thalachh: максимально достигнутая частота сердечных сокращений
9. exng: стенокардия, вызванная физической нагрузкой
    - 1: да
    - 0: нет
10. oldpeak: понижение ST, вызванное физической нагрузкой, относительно покоя
11. slp: наклон сегмента ST с максимальной нагрузкой
    - 0: повышающийся
    - 1: ровный
    - 2: нисходящий
12. caa: количество крупных сосудов (0-3), окрашенных методом флуороскопии
13. thall:
    - 0: ошибка (в исходном наборе данных соответствует NaN)
    - 1: исправленный дефект
    - 2: норма
    - 3: устраняемый дефект 
14. output:
    - 0: нет заболевания
    - 1: есть заболевание

In [101]:
heart_df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [102]:
heart_df.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

In [104]:
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [105]:
heart_df.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64